On possède un dataset de tweets, et on cherche à faire du clustering pour identifier les tweets parlant de sujets similaires.

Pour ce TP, je ne vous ai passé qu'une partie du dataset. Si votre solution marche bien, ou que vous avez besoin de plus de données, et que vous avez un meilleur pc que moi,
vous pouvez récupérer le reste du dataset ici :
    
https://github.com/fivethirtyeight/russian-troll-tweets


In [ ]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

dataset_file_1 = Path("IRAhandle_tweets_1.txt")
df = pd.read_csv(dataset_file_1)

In [ ]:
tweets_list = df["content"].values

In [ ]:
tweets_list

In [ ]:
from gensim.models import KeyedVectors
# Si vous n'avez pas déjà un word2vec entrainé : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)
model.fill_norms()

# Comment utiliser une représentation des mots pour faire du clustering de phrase ?

- Solution 1 : obtenir la representation d'un mot, puis faire votre propre representation du tweet à partir de celle la

In [ ]:
v1 = model.get_vector("Bordeaux")
print("V1_infer", v1)

- Solution 2 : utiliser la méthode fournie pour comparer directement deux ensembles de mots

In [ ]:
model.n_similarity(["I", "tweet"],["I","use","twitter"])

In [ ]:
model.n_similarity(["I", "tweet"],["I","use","facebook"])

- Solution 3 : doc2vec ou autre embedding de phrases/documents plus élaborés

### A partir du jeu de donnée, effectuer un clustering des tweets

Nettoyez votre jeu de données rapidement :

    - ne gardez que 1000 phrase pour l'instant (surtout si votre pc est lent !)
    - tokenizez chaque phrase
    - enlevez les stops words et les mots non connus par votre model pré-entrainé (non présents dans valid_tokens ci dessous)

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords  
  
valid_tokens = set(model.key_to_index.keys())
stop_words = set(stopwords.words('english'))
nb_sentence = 1_000

In [ ]:
tokenized_sentences = # TODO

Créez une matrice de similarité :

    - elle est de taille (nb_sentence, nb_sentence), ici (1000,1000)
    - la case i,j indique la distance entre la phrase i et la phrase j, calculée au moyen de model.n_similarity
    - pour l'algorithme de spectral clustering, elle doit valoir 1 si les phrases sont identiques, et 0 si les phrases sont completements différentes (ou une des phrase est vide)

In [ ]:
similarity_matrix = np.zeros((nb_sentence, nb_sentence))

# TODO : remplir la matrice

### Entrainons notre algorithme de clustering

- comme souvent en ML, une fois que l'on a fait le preprocessing adéquat, on a fait 80% du job !

In [ ]:
from sklearn.cluster import SpectralClustering
import numpy as np

cls = SpectralClustering( affinity="precomputed")
preds = cls.fit_predict(similarity_matrix)

### Visualisons tout ça

Avec une pca ?

In [ ]:
data_as_array = np.zeros((nb_sentence, 68))
for i, sen in enumerate(tokenized_sentences_int[:1000]):
    data_as_array[i, :len(sen)] = sen

In [ ]:
from sklearn.decomposition import PCA

tokenized_sentences_int 
transformed = PCA(n_components=2).fit_transform(data_as_array)

In [ ]:
plt.scatter(x=transformed[:,0], y=transformed[:,1], c=preds)

Regardons le contenu:

In [ ]:
for i in np.where(preds == 3)[0]:
    print(tokenized_sentences[i])

 Le cluster à l'air de parler de Media et politique ?
 
 Comment améliorer :
- plus de cluster
- plus que 1000 exemples en train
- semi-supervisés avec phrases d'exemples pour chaque cluster
- meilleure métrique de distance (doc2vec, transformers, ...)


Question bonus : Pourquoi à votre avis je n'ai pas voulu utiliser l'algorithme de clustering DBSCAN ici ?
- vous pouvez vérifier votre intuition en inversant la matrice de distance et en la fournissant à dbscan !